In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import nltk
import requests
import random
import math
import pathlib 
import modules.hein_scraping_functions

from modules.create_path import create_path
from modules.hein_scraping_functions import create_browser, webpage_wait, search_hein

In [2]:
# Create the paths for the data directories
input_path, work_path, intr_path, out_path, selenium_driver_path = create_path()

# Create the paths for the Chrome binary and selenium driver
chrome_binary_path = pathlib.Path("C:\\Program Files (x86)\\BraveSoftware\\Brave-Browser\\Application\\brave.exe")
selenium_driver_full_path = selenium_driver_path / "chromedriver.exe"

# Initalize the browsers that we are going to use
driver = create_browser(chrome_binary_path, selenium_driver_full_path)

driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")


In [13]:
book_df = pd.read_excel(input_path / "book_search.xlsx", sheet_name='for martin')
book_df

,Professor,Last,ID,num Coauthors,Title,search string1,search string2
0,"Adler, Matthew",Adler,1.0,1,Measuring Social Welfare: An Introduction,"""(MEASURING SOCIAL WELFARE) (ADLER)"" ~15","""(MEASURING SOCIAL WELFARE: AN INTRODUCTION) (..."
1,"Adler, Matthew",Adler,1.0,1,Well-Being And Fair Distribution: Beyond Cost-...,"""(WELL-BEING AND FAIR DISTRIBUTION) (ADLER)"" ~15","""(WELL-BEING AND FAIR DISTRIBUTION: BEYOND COS..."
2,"Adler, Matthew",Adler,1.0,1,New Foundations Of Cost-Benefit Analysis,"""(NEW FOUNDATIONS OF COST-BENEFIT ANALYSIS) (A...","""(NEW FOUNDATIONS OF COST-BENEFIT ANALYSIS) (A..."
3,"Albert, Richard",Albert,3.0,1,"Constitutional Amendments: Making, Breaking, A...","""(CONSTITUTIONAL AMENDMENTS) (ALBERT)"" ~15","""(CONSTITUTIONAL AMENDMENTS: MAKING, BREAKING,..."
4,"Boyd, William",Boyd,21.0,1,The Slain Wood: Papermaking And Its Environmen...,"""(THE SLAIN WOOD) (BOYD)"" ~15","""(THE SLAIN WOOD: PAPERMAKING AND ITS ENVIRONM..."
...,...,...,...,...,...,...,...
208,"Tomlins, Christopher",Tomlins,264.0,1,"Freedom, Bound: Law, Labor, And Civic Identity...","""(FREEDOM, BOUND) (TOMLINS)"" ~15","""(FREEDOM, BOUND: LAW, LABOR, AND CIVIC IDENTI..."
209,"Tomlins, Christopher",Tomlins,264.0,1,"Law, Labor, And Ideology In The Early American...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC..."
210,"Tomlins, Christopher",Tomlins,264.0,1,"The State And The Unions: Labor Relations, Law...","""(THE STATE AND THE UNIONS) (TOMLINS)"" ~15","""(THE STATE AND THE UNIONS: LABOR RELATIONS, L..."
211,"Zellmer, Sandra",Zellmer,287.0,2,Mississippi River Tragedies: A Century Of Unna...,"""(MISSISSIPPI RIVER TRAGEDIES) (ZELLMER)"" ~15","""(MISSISSIPPI RIVER TRAGEDIES: A CENTURY OF UN..."


In [33]:
# Load the dataset from the input directory
book_df = pd.read_excel(input_path / "book_search.xlsx")
book_df['ID'] = book_df['ID'].apply(lambda x: '{0:0>4}'.format(x))

# Check to see if the file for the alternate names data already exists.
# If it does, we only want to look for the missing observations
book_df_cur = intr_path / "_book_df.xlsx"
if book_df_cur.exists():
    print("Data already exists. Names that have already been scraped will be skipped")
    # Set the append flag to 1
    append = 1
    # Create the dataset of existing alt names.
    book_df_existing_data = pd.read_excel(book_df_cur)
    book_df_existing_data['ID'] = book_df_existing_data['ID'].apply(lambda x: '{0:0>4}'.format(x))
    # Complete a left outer join of the existing alt names and the lateral/control data to get 
    # a list of the names that we still need to scrape alt names for.
    data = pd.merge(book_df, book_df_existing_data[["ID", "Title"]], how = "outer", left_on = ["ID", "Title"], right_on = ["ID", "Title"], indicator=True)
    data = data[data['_merge'] == 'left_only']
    data = data.drop(["_merge"], axis = 1)
else:
    # Set the append flag to zero because we won't have any data to append
    append = 0
    data = book_df

data

Data already exists. Names that have already been scraped will be skipped


,Professor,Last,ID,num Coauthors,Title,search string1,search string2
50,"Garrett, Brandon",Garrett,0088,1,Too Big To Jail: How Prosecutors Compromise W...,"""( TOO BIG TO JAIL) (GARRETT)"" ~15","""( TOO BIG TO JAIL: HOW PROSECUTORS COMPROMISE..."
51,"Garrett, Brandon",Garrett,0088,1,Convicting The Innocent: Where Criminal Prose...,"""( CONVICTING THE INNOCENT) (GARRETT)"" ~15","""( CONVICTING THE INNOCENT: WHERE CRIMINAL PRO..."
52,"Garth, Bryant",Garth,0089,2,Dealing In Virtue: International Commercial Ar...,"""(DEALING IN VIRTUE) (GARTH)"" ~15","""(DEALING IN VIRTUE: INTERNATIONAL COMMERCIAL ..."
53,"Garth, Bryant",Garth,0089,2,The Internationalization Of Palace Wars: Lawye...,"""(THE INTERNATIONALIZATION OF PALACE WARS) (GA...","""(THE INTERNATIONALIZATION OF PALACE WARS: LAW..."
54,"Garth, Bryant",Garth,0089,2,Lawyers And The Rule Of Law In An Era Of Globa...,"""(LAWYERS AND THE RULE OF LAW IN AN ERA OF GLO...","""(LAWYERS AND THE RULE OF LAW IN AN ERA OF GLO..."
...,...,...,...,...,...,...,...
207,"Tomlins, Christopher",Tomlins,0264,1,"Freedom, Bound: Law, Labor, And Civic Identity...","""(FREEDOM, BOUND) (TOMLINS)"" ~15","""(FREEDOM, BOUND: LAW, LABOR, AND CIVIC IDENTI..."
208,"Tomlins, Christopher",Tomlins,0264,1,"Law, Labor, And Ideology In The Early American...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC...","""(LAW, LABOR, AND IDEOLOGY IN THE EARLY AMERIC..."
209,"Tomlins, Christopher",Tomlins,0264,1,"The State And The Unions: Labor Relations, Law...","""(THE STATE AND THE UNIONS) (TOMLINS)"" ~15","""(THE STATE AND THE UNIONS: LABOR RELATIONS, L..."
210,"Zellmer, Sandra",Zellmer,0287,2,Mississippi River Tragedies: A Century Of Unna...,"""(MISSISSIPPI RIVER TRAGEDIES) (ZELLMER)"" ~15","""(MISSISSIPPI RIVER TRAGEDIES: A CENTURY OF UN..."


In [4]:
# Search for each string and save the results to the dataset
data = data[:50]
data["Results String1"] = data["search string1"].apply(lambda x: search_hein(x, driver)).copy()
data["Results String2"] = data["search string2"].apply(lambda x: search_hein(x, driver)).copy()

<ipython-input-4-005fe5e6224b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_df_subset["Results String1"] = book_df_subset["search string1"].apply(lambda x: search_hein(x, driver)).copy()
<ipython-input-4-005fe5e6224b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  book_df_subset["Results String2"] = book_df_subset["search string2"].apply(lambda x: search_hein(x, driver)).copy()


In [6]:
# Export to Excel
data.to_excel(intr_path / "_book_df.xlsx", index = False)